# ArcFace MIL 데이터 분석 노트북\n\n세션 분할 로직 검증 및 Bag 분포 분석을 위한 노트북

In [1]:
# 세션 분할 검증\n
import pandas as pd\n
import numpy as np\n
import pickle\n
from collections import Counter\n

# CSV 로드하여 반복 번호 범위 확인\n
df_check = pd.read_csv("/workspace/MIL/data/raw/naver_ocr.csv")\n
df_check = df_check[df_check['label'] < 300]  # 실제 이미지가 있는 범위\n
\n
# 반복 번호 추출\n
df_check['rep_num'] = df_check['path'].apply(lambda x: int(x.split('_')[-1].split('.')[0]))\n

print("=== 반복 번호 범위 확인 ===")\n
print(f"최소 반복 번호: {df_check['rep_num'].min()}")\n
print(f"최대 반복 번호: {df_check['rep_num'].max()}")\n
print(f"고유 반복 번호 개수: {df_check['rep_num'].nunique()}")\n

# 세션 분할 시뮬레이션\n
def get_session(rep_num):\n
    return rep_num // 9\n

df_check['session'] = df_check['rep_num'].apply(get_session)\n
print(f"\n생성된 세션 수: {df_check['session'].nunique()}")\n
print("세션별 분포:")\n
print(df_check['session'].value_counts().sort_index())

SyntaxError: unexpected character after line continuation character (2560203075.py, line 2)

In [ ]:
# 한 작성자의 세션 구조 확인\n
writer_0 = df_check[df_check['label'] == 0]\n
print(f"작성자 0의 데이터:")\n
print(f"- 총 이미지 수: {len(writer_0)}")\n
print(f"- 반복 번호 범위: {writer_0['rep_num'].min()} - {writer_0['rep_num'].max()}")\n
print(f"- 세션 수: {writer_0['session'].nunique()}")\n
print(f"\n세션별 이미지 수:")\n
print(writer_0['session'].value_counts().sort_index())

In [ ]:
# Bag 분포 분석\n
# Train bag 로드\n
with open('/workspace/MIL/data/processed/bags/bags_arcface_margin_0.4_train.pkl', 'rb') as f:\n
    bags_train = pickle.load(f)\n

print("=== Bag 분포 분석 ===")\n
print(f"총 Bag 수: {len(bags_train)}")\n

# 라벨별 분포\n
labels = [bag['bag_label'] for bag in bags_train]\n
label_counts = Counter(labels)\n
print(f"\nPositive (단일 작성자): {label_counts[0]}")\n
print(f"Negative (복수 작성자): {label_counts[1]}")\n
print(f"비율 = 1:{label_counts[1] / label_counts[0]:.1f}")\n

# Negative bag의 위조 비율 분포\n
neg_bags = [bag for bag in bags_train if bag['bag_label'] == 1]\n
ratios = [bag.get('ratio', 'N/A') for bag in neg_bags]\n
ratio_counts = Counter(ratios)\n
print(f"\nNegative Bag의 위조 비율 분포:")\n
for ratio, count in sorted(ratio_counts.items()):\n
    print(f"  {ratio}: {count}개")

In [ ]:
# 문서별 Bag 생성 수 분석\n
doc_bag_counts = Counter([bag['doc'] for bag in bags_train])\n
bag_counts_dist = Counter(doc_bag_counts.values())\n

print("=== 문서별 Bag 생성 분포 ===")\n
print(f"총 문서 수: {len(doc_bag_counts)}")\n
print(f"\n문서당 Bag 수 분포:")\n
for bag_count, doc_count in sorted(bag_counts_dist.items()):\n
    print(f"  {bag_count}개 bag: {doc_count}개 문서")\n

# 평균 및 통계\n
bag_counts_list = list(doc_bag_counts.values())\n
print(f"\n평균 bag/문서: {np.mean(bag_counts_list):.2f}")\n
print(f"최소 bag/문서: {np.min(bag_counts_list)}")\n
print(f"최대 bag/문서: {np.max(bag_counts_list)}")